In [1]:
import numpy as np
from ase import Atoms
from copy import deepcopy
from itertools import combinations
import pandas as pd
from ase.io import read, write
from configurations import *
import sys
#sys.path.insert(0,'../pycp2k')
#sys.path.insert(0,'../python-utils')
from ase.build import molecule, graphene_nanoribbon
from ase.visualize import view

In [2]:
cn = molecule('CN')
cn.positions

array([[ 0.      ,  0.      , -0.611046],
       [ 0.      ,  0.      ,  0.523753]])

In [3]:
cn.positions
cn.set_cell([10,10,10])
cn.pbc = True
cn.center()
cn.rotate(v='x',a=np.pi/2., center='COM')

In [4]:
view(cn)

In [5]:
lattice = build_lattice(4, motif=cn,lattice_constant=3)

In [6]:
lattice

[Atoms(symbols='CN', pbc=True, cell=[10.0, 10.0, 10.0], initial_magmoms=...),
 Atoms(symbols='CN', pbc=True, cell=[10.0, 10.0, 10.0], initial_magmoms=...),
 Atoms(symbols='CN', pbc=True, cell=[10.0, 10.0, 10.0], initial_magmoms=...),
 Atoms(symbols='CN', pbc=True, cell=[10.0, 10.0, 10.0], initial_magmoms=...)]

In [7]:
conf = Atoms()

In [8]:
for point in lattice:
    conf.extend(point)

In [9]:
view(conf)

In [10]:
confs = create_configurations(lattice,axis = 'z', angle = 180.0)

In [11]:
confs

{'AAAA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'AAAC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'AACA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'AACC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'ACAA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'ACAC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'ACCA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'ACCC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CAAA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CAAC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CACA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CACC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CCAA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CCAC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CCCA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CCCC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...)}

In [12]:
view(confs.values())

In [18]:
bmim_pf6_opt = read('./opt_coords_bigbox.xyz')
bmim_pf6_opt.cell = [44.20800018, 42.54000092, 50.]
bmim_pf6_opt.center()

In [19]:
view(bmim_pf6_opt)

In [20]:
pf6 = bmim_pf6_opt[0:7]; bmim = bmim_pf6_opt[7:32]; electrode = bmim_pf6_opt[32:]

In [21]:
pair = pf6+bmim
eq_dist_ion_pair_electrode = abs(electrode.get_center_of_mass()-pair.get_center_of_mass())[2]
eq_dist_ion_pair_electrode

4.4099961471344038

In [22]:
eq_dist_ion_pair = np.linalg.norm(bmim.get_center_of_mass() - pf6.get_center_of_mass())
eq_dist_ion_pair

5.0451744679708543

In [15]:
view(pair)

In [23]:
pair.rotate('x',a=np.pi/2,center='COM')

In [17]:
view(pair)

In [19]:
# pair.rotate('x',a=np.pi/9.,center='COM')

In [20]:
# view(pair)

In [24]:
electrode.center()
pair.center()
pair.translate([0.0,0.0,-eq_dist_ion_pair_electrode])

In [26]:
view(pair)

In [47]:
vector = 'x'
rotation_angle = 180.0
separation = 15.0
pairs = 2
lshift = -8.0
rshift = 8.0
add_electrode = True
tailored_box = True
z_len = 50.

In [48]:
generated_electrode = False
cell = bmim_pf6_opt.cell
lattice = build_lattice(pairs, motif=pair,lattice_constant=separation)
l_confs = create_configurations(lattice,vector,rotation_angle)
######
for key in l_confs.keys():
    if tailored_box:
        while not generated_electrode:
            electrode = generate_electrode(l_confs[key], lattice_constant=separation, z_len=z_len)
            cell = electrode.cell
            generated_electrode = True
    l_confs[key].set_cell(cell)
    l_confs[key].center(axis=(0,1))
    if lshift != 0.0:
        l_confs[key].center(axis=2)
        l_confs[key].translate([0.0,0.0,lshift])
        if add_electrode and too_close(l_confs[key]+electrode,1.0):
            (l_confs[key]+electrode).write('lhs_'+key+'_positions.vasp')
            print('LHS ions are too close to the electrode')
            sys.exit()

lattice = build_lattice(pairs, motif=pair,lattice_constant=separation)
r_confs = create_configurations(lattice,vector,rotation_angle)
for key in r_confs.keys():
    r_confs[key].set_cell(cell)
    r_confs[key].rotate('y',a=np.pi,center='COM')
    r_confs[key].center(axis=(0,1,2))
    r_confs[key].translate([0.0,0.0,eq_dist_ion_pair_electrode])
    if rshift != 0.0:
        r_confs[key].center(axis=2)
        r_confs[key].translate([0.0,0.0,rshift])
        if add_electrode and too_close(r_confs[key]+electrode,1.0):
            (r_confs[key]+electrode).write('rhs_'+key+'_positions.vasp')
            print('RHS ions are too close to the electrode')
            sys.exit()

In [49]:
full_confs = []
for  l_key in l_confs.keys():
    for  r_key in r_confs.keys():
        box = l_confs[l_key] + r_confs[r_key]
        if add_electrode:
            box.extend(electrode)
        box.write('L-'+l_key+'-R-'+r_key+'-positions.cif')
        full_confs.append(box)

In [50]:
view(full_confs)

In [669]:
from ase.build import *
from copy import deepcopy

In [670]:
rocksalt = bulk('NaCl', crystalstructure='rocksalt', a=5.64, orthorhombic=True)

In [671]:
supercell1 = make_supercell(rocksalt,[[2.0,0.0,0.0],[0.0,2.0,0.0],[0.0,0.0,1.0]])

In [672]:
supercell2 = deepcopy(supercell1)

In [673]:
supercell2.rotate(a=180.0,v='y',center='COP')

In [674]:
supercell1.get_cell()

array([[ 7.97616449,  0.        ,  0.        ],
       [ 0.        ,  7.97616449,  0.        ],
       [ 0.        ,  0.        ,  5.64      ]])

In [675]:
supercell2.get_cell()

array([[ 7.97616449,  0.        ,  0.        ],
       [ 0.        ,  7.97616449,  0.        ],
       [ 0.        ,  0.        ,  5.64      ]])

In [676]:
from configurations import *

In [677]:
sheet = graphene_nanoribbon(1,1,sheet=True)

In [678]:
sheet.get_cell()

array([[ 2.13      ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  2.45951215]])

In [679]:
sheet.get_cell()[2][2]

2.4595121467478056

In [680]:
np.ceil(supercell1.get_cell()[0][0] / sheet.get_cell()[2][2])

4.0

In [681]:
sheet = graphene_nanoribbon(4,4,sheet=True)

In [682]:
sheet.get_cell()

array([[ 8.52      ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  9.83804859]])

In [683]:
electrode = generate_electrode(supercell1, 0.0, x_in=supercell1.get_cell()[0][0], y_in=supercell1.get_cell()[0][0], z_len=20.0)

In [684]:
electrode_cell = electrode.get_cell()

In [685]:
supercell1.set_cell([electrode_cell[0][0],electrode_cell[1][1],supercell1.get_cell()[2][2]],scale_atoms=True)
supercell2.set_cell([electrode_cell[0][0],electrode_cell[1][1],supercell1.get_cell()[2][2]],scale_atoms=True)

In [686]:
supercell1.set_cell(electrode.get_cell())
supercell2.set_cell(electrode.get_cell())

In [687]:
supercell1.center()
supercell2.center()

In [688]:
supercell1.translate([0.0,0.0,-4.8])
supercell2.translate([0.0,0.0,4.8])
# supercell2.translate([0.25*electrode.get_cell()[0][0],0.0,0.0])
supercell2.translate([0.5*electrode.get_cell()[0][0],0.25*electrode.get_cell()[1][1],0.0])

In [689]:
system = electrode+supercell1+supercell2

In [690]:
system.set_cell(electrode.get_cell())

In [691]:
# view(system)

In [692]:
system.write('system.cif')
system.write('system.xyz')

In [693]:
!VESTA system.cif system.xyz